\begin{center}
Gabe Morris
\end{center}

In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Latex

plt.style.use('maroon.mplstyle')

s, t = sp.symbols('s t')

display_latex = lambda text: display(Latex(text))

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 6643 & Homework 7 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 9.23
## Given
A certain factory contains a heavy rotating machine that causes the factory floor to vibrate. We want to operate another piece of equipment nearby, and we measure the amplitude of the floor’s motion at that point to be 0.01 m. The mass of the equipment is 1500 kg and its support has a stiffness of $k=2\cdot10^4\,\frac{N}{m}$ and a damping ratio of $\zeta=0.04$.

## Find
Calculate the maximum force that will be transmitted to the equipment at resonance.

## Solution
The free response of this system is,

$m\ddot{x}+c\dot{x}+kx=0$

We need to calculate $c$.

In [2]:
# Getting the damping constant using the damping ratio
zeta, m, k = 0.04, 1500, 2e4
c = 2*zeta*sp.sqrt(m*k)
c

438.178046004133

In [3]:
x = sp.Function('x')(t)
eq = sp.Eq(m*x.diff(t, 2) + c*x.diff(t) + k*x, 0)
eq

Eq(20000.0*x(t) + 438.178046004133*Derivative(x(t), t) + 1500*Derivative(x(t), (t, 2)), 0)

In [4]:
for root in sp.roots(m*s**2 + c*s + k):
    display_latex(f'${sp.latex(root)}$')

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Resonance occurs when the excitation frequency is equivalent to the natural frequency of the system ($\omega=\omega_n=3.649\frac{rad}{s}$). The relationship for this system is described in section 9.3.5 in the book.

$\frac{F_t(s)}{Y(s)}=\frac{(cs+k)ms^2}{ms^2+cs+k}$

In [5]:
expr = (c*s + k)*m*s**2/(m*s**2 + c*s + k)
expr

s**2*(657267.069006199*s + 30000000.0)/(1500*s**2 + 438.178046004133*s + 20000.0)

In [6]:
expr_tw = expr.subs(s, sp.I*3.64856136031724)
expr_tw

-5.20624749699639e-6*(32.0000000000182 - 1598.71948758999*I)*(30000000.0 + 2398079.23138498*I)

In [7]:
sp.Abs(expr_tw)*0.01

2505.46545288473

## Answer
$F_{max}=2500\,N$